# Packages

In [0]:
import dataiku

import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from edf_commons.modelling import preprocess_data

import tempfile
from datetime import datetime
import shutil
import pickle
import os

# Global variables

In [0]:
project = dataiku.Project()
variables = project.get_variables()
MODEL_FOLDER_ID = variables["standard"]["df_model_folder_id"]
MODELS_DATA_FOLDER = dataiku.Folder(MODEL_FOLDER_ID)

# Input

In [0]:
tweets_eval = dataiku.Dataset("tweets_eval")
eval_df = tweets_eval.get_dataframe()

In [0]:
# Get the Logistic Regression model
dl_path = MODELS_DATA_FOLDER.list_paths_in_partition()[-1]
lr_name = model_path.split("/")[-1].split(".pkl")[0]

# load latest TFIDF model
with tempfile.TemporaryDirectory() as temp_directory_name:

    local_file_path = temp_directory_name + "/" + lr_name

    # Copy file from remote to local
    with MODELS_DATA_FOLDER.get_download_stream(lr_path) as f_remote, open(local_file_path,'wb') as f_local:
        shutil.copyfileobj(f_remote, f_local)

    # Load the pipeline
    with open(local_file_path, 'rb') as file:
        lr_model = pickle.load(file)